In [3]:
from intelhex import IntelHex
import csv

class HexTruth:
    def __init__(self, csvFilename, hexFilename=None ):
        if hexFilename is None: hexFilename = csvFilename.split(".")[0]+ ".hex"
        self.csvFilename = csvFilename
        self.hexFilename = hexFilename
        self.ih= IntelHex()
        
    def writeFromIntelHex(self): #Takes the IntelHex object and writes it to a hex file.
        f = open(self.hexFilename, 'w');
        self.ih.write_hex_file(f)
        f.close()
    
    def importCSV(self): #Opens a given csv filename and returns it's values as a list of lists.
        #list of lists = rows of columns
        self.csvTruth = []
        with open(self.csvFilename, newline='') as csvfile:
            csvReadingMachine = csv.reader(csvfile, delimiter=',', quotechar='"')
            for row in csvReadingMachine:
                self.csvTruth.append(row)
        return self.csvTruth
    
    def parseCSV(self, csvTruth, rangeIn = [0,8], rangeOut = [10]): #Takes list of lists and converts it into a 
        #dict object for IntelHex to use. (ih.fromdict(hexDict))
        self.hexDict = {};
        for line in csvTruth:
            if "#" in str(line):continue
            [addr, dat] = str(line).split("''")
            rangeIn = [0, addr.count(',') -1]
            if dat.count(",") is 1: rangeOut = [rangeIn[-1] + 2]
            else:rangeOut = [rangeIn[-1] + 2, rangeIn[-1] + 1 + dat.count(",")]
                       
            binaryString = ""
            for j in rangeIn:                
                binaryString+=str(line[j])
            address = (int(binaryString,2))
            binaryString = ""
            for j in rangeOut:
                binaryString+=str(line[j])
            data = (int(binaryString,2))
            self.hexDict[address] = data
        return self.hexDict

In [6]:
fileName = input("CSV filename? (Is file in the same directory as this program?)")
tt = HexTruth(fileName)
cTruth = tt.importCSV()
newDict = tt.parseCSV(cTruth)
tt.ih.fromdict(newDict)
tt.writeFromIntelHex()
print("Hex file generated")

CSV filename? (Is file in the same directory as this program?)b2TruthTable.csv


/home/jake/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:13: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


In [3]:
class TruthWriter:
    def __init__(self, addressSize, dataSize, logicType, logicRange = 8, selectorRange = 3):
        self.addressSize=addressSize
        self.dataSize=dataSize
        self.logicType = logicType
    def genDict(self):
        logicDict = {}
        for i in range(0, 2**addressSize-1):
            bString = "{0:0" + str(addressSize) + "b}"
            bString.format(i)
            
        

In [47]:
addressSize = 8
logicDict = {}
logic = 'and'
for i in range(0, 2**addressSize): #builds the logic
    address = "{0:0" + str(addressSize) + "b}"
    runningLogic = int(address.format(i)[0])
    for e in address.format(i):
        if logic is 'and': runningLogic = runningLogic & int(e)
        if logic is 'or': runningLogic = runningLogic | int(e)
    logicDict[address.format(i)] = runningLogic
    

In [48]:
logicDict


{'00000000': 0,
 '00000001': 0,
 '00000010': 0,
 '00000011': 0,
 '00000100': 0,
 '00000101': 0,
 '00000110': 0,
 '00000111': 0,
 '00001000': 0,
 '00001001': 0,
 '00001010': 0,
 '00001011': 0,
 '00001100': 0,
 '00001101': 0,
 '00001110': 0,
 '00001111': 0,
 '00010000': 0,
 '00010001': 0,
 '00010010': 0,
 '00010011': 0,
 '00010100': 0,
 '00010101': 0,
 '00010110': 0,
 '00010111': 0,
 '00011000': 0,
 '00011001': 0,
 '00011010': 0,
 '00011011': 0,
 '00011100': 0,
 '00011101': 0,
 '00011110': 0,
 '00011111': 0,
 '00100000': 0,
 '00100001': 0,
 '00100010': 0,
 '00100011': 0,
 '00100100': 0,
 '00100101': 0,
 '00100110': 0,
 '00100111': 0,
 '00101000': 0,
 '00101001': 0,
 '00101010': 0,
 '00101011': 0,
 '00101100': 0,
 '00101101': 0,
 '00101110': 0,
 '00101111': 0,
 '00110000': 0,
 '00110001': 0,
 '00110010': 0,
 '00110011': 0,
 '00110100': 0,
 '00110101': 0,
 '00110110': 0,
 '00110111': 0,
 '00111000': 0,
 '00111001': 0,
 '00111010': 0,
 '00111011': 0,
 '00111100': 0,
 '00111101': 0,
 '001111

In [27]:
bString[0]

'1'

In [24]:
bString.split("d")

['1111111111111110']

In [2]:
csvFileName = 'b2TruthTable.csv'


        

In [4]:
from intelhex import IntelHex
ih = IntelHex()
csvList = importCSV("b2TruthTable.csv")
hexDict = parseCSV(csvList, rangeIn = [0,1], rangeOut = [3])
ih.fromdict(hexDict)
ih.dump()
writeFromIntelHex(ih)

0000  00 00 00 01 -- -- -- -- -- -- -- -- -- -- -- --  |....            |


/home/jake/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  app.launch_new_instance()
